In [19]:
import requests
import pytz
from datetime import datetime, timedelta
import json
import os
import time

In [20]:
session = requests.session()

In [21]:
# EE VMS User Credentials:
base       = "login"
branding   = "" # will populate at a later stage
username   = ""
password   = ""
# EE VMS API Key:
api_key    = "" # not mandatory for API V1/V2
# Start and end timestamp:
timestamp = (datetime.now(pytz.timezone('UTC')).strftime('%Y%m%d%H%M%S.000')) # UTC Timestamp
tz = pytz.timezone('Europe/Amsterdam') # Amsterdam TimeZone
start_time = timestamp # Start TimeStamp
end_time   = (datetime.now(tz=tz).strftime('%Y%m%d%H%M%S.000')) # End_Time = start_time + 1 hour

In [22]:

url = "https://" + base + ".eagleeyenetworks.com/g/aaa/authenticate"
payload = json.dumps({
    'username': username,
    'password': password
})
headers = {
    'content-type': 'application/json',
    'authorization': api_key
}
response = session.request("POST", url, data=payload, headers=headers)
token = response.json()['token']

In [23]:
# Login Eagleeyenetworks
url = "https://" + base + ".eagleeyenetworks.com/g/aaa/authorize"
payload = json.dumps({'token': token})
headers = {
    'content-type': 'application/json',
    'authorization': api_key
}
response = session.request("POST", url, data=payload, headers=headers)
auth_key = session.cookies.get_dict()["auth_key"]
branding = response.json()["active_brand_subdomain"]

In [24]:
camera_ids = []
url = "https://"+branding+".eagleeyenetworks.com/g/device/list"
payload = ""
headers = {'authorization': api_key}
response = session.request("GET",url)
if response.status_code == 200:
    for device in response.json():
        if device[3] == "camera":
            if len(camera_ids) < 32:
                camera_ids.append(device[1])
    for id in camera_ids:
        url1 = "https://"+branding+".eagleeyenetworks.com/g/device"
        url2 = "https://"+branding+".eagleeyenetworks.com/g/device/rtsp"
        params = {"id": id}
        r = session.request("GET",url1,params=params)
        re = session.request("GET",url2,params=params)
        if r.status_code == 200:
            print("/g/device", r.status_code, params)
        else:
            print("not getting 200 but: ", r.status_code, params)     
        if re.status_code == 200:
            print("/g/device/rtsp", re.status_code, params)
            if re.json()["preview_url"] == "" or \
               re.json()["video_url"] == "":
                    print("get device rtsp",id, re.status_code)
        else:
            print("Get Camera RTSP error", params, re.status_code )        
else:       
    print("response 01", response.status_code)
print("End of the script", response.status_code)

/g/device 200 {'id': '100c96fc'}
Get Camera RTSP error {'id': '100c96fc'} 500
/g/device 200 {'id': '100207f5'}
/g/device/rtsp 200 {'id': '100207f5'}
/g/device 200 {'id': '100b1529'}
/g/device/rtsp 200 {'id': '100b1529'}
/g/device 200 {'id': '1007ccd8'}
/g/device/rtsp 200 {'id': '1007ccd8'}
/g/device 200 {'id': '1008a415'}
/g/device/rtsp 200 {'id': '1008a415'}
/g/device 200 {'id': '10096def'}
/g/device/rtsp 200 {'id': '10096def'}
/g/device 200 {'id': '1006ab89'}
/g/device/rtsp 200 {'id': '1006ab89'}
/g/device 200 {'id': '100fbe9f'}
/g/device/rtsp 200 {'id': '100fbe9f'}
/g/device 200 {'id': '1008aa93'}
/g/device/rtsp 200 {'id': '1008aa93'}
/g/device 200 {'id': '100de948'}
/g/device/rtsp 200 {'id': '100de948'}
End of the script 200
